In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [94]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(
            in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

In [101]:
tensor = torch.randn(1,1,512,512)
tensor.shape

torch.Size([1, 1, 512, 512])

In [104]:
block = vgg_block(1,1,64)
X = tensor
for layer in block:
    X = layer(X)
    print(layer.__class__.__name__, 'shape \t:',X.shape)

print('-----------------')

block2 = vgg_block(1,64,128)
for layer in block2:
    X = layer(X)
    print(layer.__class__.__name__, 'shape \t:',X.shape)

Conv2d shape 	: torch.Size([1, 64, 512, 512])
ReLU shape 	: torch.Size([1, 64, 512, 512])
MaxPool2d shape 	: torch.Size([1, 64, 256, 256])
-----------------
Conv2d shape 	: torch.Size([1, 128, 256, 256])
ReLU shape 	: torch.Size([1, 128, 256, 256])
MaxPool2d shape 	: torch.Size([1, 128, 128, 128])


In [106]:
conv_arch = ((1,64),(1,128),(2,256),(2,512),(2,512))
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    for(num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(
            num_convs, in_channels, out_channels))
        in_channels = out_channels
        print("看这里"+str(out_channels))
    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(out_channels*16*16,4096),nn.ReLU(), # 这个7*7 很 妙啊 是怎么确定的
        nn.Dropout(0.5), nn.Linear(4096,4096),nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096,10))

In [107]:
net = vgg(conv_arch)

看这里64
看这里128
看这里256
看这里512
看这里512


In [108]:
X = torch.randn(size=(1, 1, 512, 512))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 256, 256])
Sequential output shape:	 torch.Size([1, 128, 128, 128])
Sequential output shape:	 torch.Size([1, 256, 64, 64])
Sequential output shape:	 torch.Size([1, 512, 32, 32])
Sequential output shape:	 torch.Size([1, 512, 16, 16])
Flatten output shape:	 torch.Size([1, 131072])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])
